# Assignment
1. Determine the five most common journals and the total articles for each. 
2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import combinations
from hunspell import Hunspell
import re
import time
import os

In [2]:
def clean_title(s):
    cs = s.strip().lower()
    cs = re.sub(r'&', 'and', cs) #replace ampersand with and
    cs = re.sub(r'[^a-zA-Z0-9]', ' ', cs) #replace non alpha numeric with space

    cs = re.sub(r'\A\s*(j|jnl)(\s+)',' journal ',cs) #replace j and jnl at beginning of string with journal
    cs = re.sub(r'\s+(j|jnl)(\s+)',' journal ',cs) #replace j and jnl not at beginning of string with journal
    
    cs = re.sub(r'\s+chem\s+',' chemistry ',cs) #replace chem with chemistry middle of word
    cs = re.sub(r'\s+chem\Z',' chemistry ',cs) #replace chem with chemistry end of word
    
    cs = re.sub(r'\s+biol\s+',' biological ',cs) #replace biol in middle of word with biological
    cs = re.sub(r'\s*biol\Z',' biology ',cs) #replace biol at end of word with biology
    
    cs = re.sub(r'\s*biol open\s*',' biology open ',cs) #replace biol open with biology open
    
    cs = re.sub(r'\s+med\s+',' medicinal ',cs) #replace med with medical
    cs = re.sub(r'\s*1\s*',' one ',cs) #replace 1 with one
    cs = re.sub(r'\s*public library of science\s*',' plos ',cs) #replace public library of science with plos
    
    cs = re.sub(r'\s+immunol\s+',' immunology ',cs) #replace immunol with immunology middle of string
    cs = re.sub(r'\s+immunol\Z',' immunology ',cs) #replace immunol with immunology end of string
    cs = re.sub(r'\Aimmunol\s+',' immunology ',cs) #replace immunol with immunology beginning of string
    
    cs = re.sub(r'\s+org\s+',' organic ',cs) #replace org with organic
    cs = re.sub(r'\s*eur\s+',' european ',cs) #replace eur with european
    
    cs = re.sub(r'\s+mol\s+',' molecular ',cs) #replace mol with molecular middle of string
    cs = re.sub(r'\Amol\s+',' molecular ',cs) #replace mol with molecular beginning of string

    
    cs = re.sub(r'\s+sci\s+',' science ',cs) #replace sci with science
    
    
    cs = re.sub(r'\s+bioeth\s+',' bioethics ',cs) #replace bioeth with bioethics
    cs = re.sub(r'\s+bioeth\Z',' bioethics',cs) #replace bioeth with bioethics
    
    cs = re.sub(r'\s*acta d\s*',' acta crystallographica d ',cs) #replace acta d  with acta crystallographica d
    cs = re.sub(r'\s*acta f\s*',' acta crystallographica f ',cs) #replace acta f  with acta crystallographica f
    cs = re.sub(r'\s+section\s+','',cs) #remove section
    cs = re.sub(r'plosone',' plos one ',cs) #replace plosone with plos one
    
    cs = re.sub(r'.*national academy of sciences',' pnas ',cs) #proceedings of the national academy of sciences to pnas
    cs = re.sub(r'.*pnas.*','pnas',cs) #reduce anything with pnas in it to only pnas
    
    cs = re.sub(r'j clin microbiol',' journal of clinical microbiology ',cs) 
    
    cs = re.sub(r'\s+dis\s+',' disease ',cs) #dis to disease in middle of string
    cs = re.sub(r'\s+dis\Z',' disease',cs) #dis to disease at end of string
    
    cs = re.sub(r'\s+infect\s+',' infectious ',cs) #infect to infectious in middle of string
    cs = re.sub(r'\s+infect\Z',' infections ',cs) #infect to infections at end of string

    cs = re.sub(r'journal of acquired immune deficiency syndromes',' jaids ',cs) #proceedings of the national academy of sciences to pnas
    cs = re.sub(r'.*jaids.*',' jaids ',cs) #reduce anything with jaids in it to only jaids
    
    cs = re.sub(r'\s+virol\Z',' virology ',cs) #virol at end of input to virology

    cs = re.sub(r'\s+pharm\Z',' pharmacology ',cs) #pharm at end of word to pharmacology
    
    cs = re.sub(r'\s+biotechnol\s+',' biotechnology ',cs)# biotechnol to biotechnology in middle of word
    cs = re.sub(r'\s+biotechnol\Z',' biotechnology ',cs)# biotechnol to biotechnology at end of word
    cs = re.sub(r'\Abiotechnol\s+',' biotechnology ',cs)# biotechnol to biotechnology at beginning of word
    
    cs = re.sub(r'\s+biochem\s+',' biochemical ',cs)# biochem to biochemical in middle of word
    cs = re.sub(r'\Abiochem\s+',' biochemical ',cs)# biochem to biochemical at beginning of word
    
    cs = re.sub(r'\Aimplement\s+',' implementation ',cs)# implement to implementation at beginning of word

    cs = re.sub(r'\s+transm\s+',' transmitted ',cs) #transm to transmitted
    
    cs = re.sub(r'\Aam\s+',' American ', cs)
    
    cs = re.sub(r'\Adev\s+',' developing ', cs)
    
    cs = cs.strip().lower()
    return cs

In [3]:
wt = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")

# Clean "Journal title" Column

In [4]:
#strip, lower case  Journal titles
wt['Journal title'] = wt['Journal title'].str.strip().str.lower().fillna('')

In [5]:
wt['cleansed_journal_title'] = wt['Journal title'].apply(clean_title)
wt

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),cleansed_journal_title
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,£0.00,psychological medicine
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,biomacromolecules
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,journal medicinal chemistry
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,journal medicinal chemistry
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,£685.88,journal organic chemistry
5,PMC3579457,ACS,journal of medicinal chemistry,Comparative Structural and Functional Studies ...,£2392.20,journal of medicinal chemistry
6,PMC3709265,ACS,journal of proteome research,Mapping Proteolytic Processing in the Secretom...,£2367.95,journal of proteome research
7,23057412 PMC3495574,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,£649.33,molecular pharmacology
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,acs chemical biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59,acs chemical biology
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,acs chemical biology,Chemical proteomic analysis reveals the drugab...,£1294.78,acs chemical biology


In [6]:
#get every combination of unique journal titles (as strings) and put into a dataframe

combos = pd.DataFrame(list(combinations(wt['cleansed_journal_title'].unique().astype(str),2)), columns=['title1','title2'])
combos.describe()

,title1,title2
count,365085,365085
unique,854,854
top,psychological medicine,pediatr infectious disease j
freq,854,854


In [7]:
def fuzz_ratio_ignore(s1, s2):
    # use this to manually ignore and include some combos. Can go further with this with more time.
    combos_to_ignore = [
        {'acta crystallographica d', 'acta crystallographica f'},
        {'analytical chemistry', 'analytical biochemistry'},
        {'journal of medical ethics', 'journal of medical genetics'},
        {'international journal of cardiology', 'international journal of parasitology'},
        {'british journal of haematology', 'british journal of pharmacology'}
    ]
    
    combos_to_include = [
        {'lancet','the lancet'}
    ]
    
    for combo in combos_to_ignore:
        if s1 in combo and s2 in combo:
            return 0
    for combo in combos_to_include:
        if s1 in combo and s2 in combo:
            return 100
    return fuzz.ratio(s1, s2)

In [8]:
# give a fuzz ratio to every combo of journal titles
combos['fuzz_ratio'] = combos.apply(lambda s: fuzz_ratio_ignore(s['title1'], s['title2']), axis=1)
combos.head()

,title1,title2,fuzz_ratio
0,psychological medicine,biomacromolecules,36
1,psychological medicine,journal medicinal chemistry,49
2,psychological medicine,journal organic chemistry,38
3,psychological medicine,journal of medicinal chemistry,46
4,psychological medicine,journal of proteome research,28


In [9]:
##### find the highest fuzz ratio level where a combo is not the same journal
combos[combos['fuzz_ratio']==91]

# the faseb journal and the febs journal are different journals
# Journal title pairings with a fuzz ratio above 91 refer to the same journal

,title1,title2,fuzz_ratio
10358,journal of immunology,the journal of immunology,91
18156,journal of physiology,the journal of physiology,91
18988,the journal of neurophysiology,the journal of physiology,91
19563,american journal of psychiatry,asian journal of psychiatry,91
93067,journal of endocrinology,journal of neuroendocrinology,91
254349,the faseb journal,the febs journal,91
274134,acta crystallography d,acta crystallographica d,91
279068,american journal of medical genetics,american journal of medical genetics part a,91
279596,the journal of pathology,journal of pathology,91


In [10]:
def unique_strings_list(series):
    return list(series.dropna().unique().astype(str))

def fuzz_groups(usl, auto_accept_fuzz_ratio):
    ''' Takes a unique strings list and a minimum fuzz ratio, returns groups of strings as list of lists.
    Within each group, fuzz.ratio(s1, s2) returns a number that is at least the min_fuzz_ratio when passed the first string of the group and any other string '''
    
    groups = [] # groups of like strings list
    
    '''For each string in usl (s1), find its fuzzy ratio with all the remaining strings (s2) below it . 
    If a match is found (high enough fuzzy ratio) and it is the first match found, 
        append a new list inside the groups list containing both strings (s1 and s2)
    If a match is found and it is not the first match, 
        append only the second string (s2) to the list that already contains the first journal title (which will be the list at groups[-1])
    Remove every matched string from usl (by changing value to None)'''
    
    for s1 in usl: 
        match = False
        for s2 in usl[usl.index(s1)+1:]:
            
            #if match is found (fuzz.ratio(s1, s2) is at least the auto_accept_fuzz_ratio)
            if fuzz_ratio_ignore(s1, s2) >= auto_accept_fuzz_ratio: 
                
                if not match: #if first match, append new list containing s1 to groups
                    groups.append([s1])
                    
                match = True
                groups[-1].append(s2) # append s2 to group list
                usl[usl.index(s2)] = None #s2 matched so remove it from usl list
                
        if match == True: #remove s1 from usl list if it had a match
            usl[usl.index(s1)] = None
    
    usl_remaining = [i for i in usl if i]
    return groups, usl_remaining
    

In [11]:
usl = unique_strings_list(wt['cleansed_journal_title'])

journal_groups, _ = fuzz_groups(usl, 92)

journal_groups

[['psychological medicine', 'pyschological medicine'],
 ['journal medicinal chemistry', 'journal of medicinal chemistry'],
 ['journal of biological chemistry',
  'journal biological chemistry',
  'the journal of biological chemistry'],
 ['journal of immunology', 'journal immunology'],
 ['journal of neurophysiology', 'the journal of neurophysiology'],
 ['american journal of psychiatry', 'americal journal of psychiatry'],
 ['antimicrobial agents and chemotherapy',
  'antimicrobial agfents and chemotherapy',
  'antimicobial agents and chemotherapy'],
 ['journal of clinical microbiology', 'jounral of clinical microbiology'],
 ['the american journal of human genetics',
  'american journal of human genetics',
  'the americal journal of human genetics'],
 ['biological chemistry', 'biologicial chemistry'],
 ['mcp  molecular and cellular proteomics',
  'molecular and cellular proteomics'],
 ['biochemical journal', 'biochemical journals'],
 ['bmc  public health', 'bmc public health'],
 ['bmc gen

In [12]:
def spell_check(s):
    spelling = True
    h = Hunspell()
    for word in s.split():
        if not ((h.spell(word)) or h.spell(word.title())or h.spell(word.upper())):
            spelling = False
    return spelling

In [13]:
title_cleanse_dict = {}
for group in journal_groups:
    spelling = False
    for title in group:
        if spell_check(title):
            title_cleanse_dict.update({d_title: title for d_title in group})
            spelling = True
    if not spelling:
        title_cleanse_dict.update({d_title: group[0] for d_title in group})
                                  
title_cleanse_dict

{'psychological medicine': 'psychological medicine',
 'pyschological medicine': 'psychological medicine',
 'journal medicinal chemistry': 'journal of medicinal chemistry',
 'journal of medicinal chemistry': 'journal of medicinal chemistry',
 'journal of biological chemistry': 'the journal of biological chemistry',
 'journal biological chemistry': 'the journal of biological chemistry',
 'the journal of biological chemistry': 'the journal of biological chemistry',
 'journal of immunology': 'journal immunology',
 'journal immunology': 'journal immunology',
 'journal of neurophysiology': 'the journal of neurophysiology',
 'the journal of neurophysiology': 'the journal of neurophysiology',
 'american journal of psychiatry': 'american journal of psychiatry',
 'americal journal of psychiatry': 'american journal of psychiatry',
 'antimicrobial agents and chemotherapy': 'antimicrobial agents and chemotherapy',
 'antimicrobial agfents and chemotherapy': 'antimicrobial agents and chemotherapy',
 

In [14]:
wt['cleansed_journal_title'] = wt['cleansed_journal_title'].apply(lambda x: title_cleanse_dict.get(x, x))
wt

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),cleansed_journal_title
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,£0.00,psychological medicine
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,biomacromolecules
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,journal of medicinal chemistry
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,journal of medicinal chemistry
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,£685.88,journal organic chemistry
5,PMC3579457,ACS,journal of medicinal chemistry,Comparative Structural and Functional Studies ...,£2392.20,journal of medicinal chemistry
6,PMC3709265,ACS,journal of proteome research,Mapping Proteolytic Processing in the Secretom...,£2367.95,journal of proteome research
7,23057412 PMC3495574,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,£649.33,molecular pharmacology
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,acs chemical biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59,acs chemical biology
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,acs chemical biology,Chemical proteomic analysis reveals the drugab...,£1294.78,acs chemical biology


# Make cost values usable

In [16]:
def clean_cost(s):
    cs = re.sub(r'[^0-9.]', '', s)
    cs = cs.strip()
    return float(cs)

In [17]:
wt['cleansed_cost'] = wt['COST (£) charged to Wellcome (inc VAT when charged)'].apply(clean_cost)

# 1. Determine the five most common journals and the total articles for each. 

In [18]:
wt['cleansed_journal_title'].value_counts()

plos one                                               208
the journal of biological chemistry                     71
pnas                                                    39
nucleic acid research                                   29
neuroimage                                              29
plos genetics                                           24
plos pathogens                                          24
plos negected tropical diseases                         21
nature communications                                   19
human molecular genetics                                19
bmc  public health                                      15
movement disorders                                      15
journal of neuroscience                                 15
brain                                                   14
biochemical journals                                    14
development cell                                        13
journal of medicinal chemistry                          

# 2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [19]:
data = [] 
for journal in wt['cleansed_journal_title'].unique():
    name = journal
    mean = wt[wt['cleansed_journal_title']==journal].cleansed_cost.mean()
    median = wt[wt['cleansed_journal_title']==journal].cleansed_cost.median()
    stdev = wt[wt['cleansed_journal_title']==journal].cleansed_cost.std()
    
    data.append([name, mean, median, stdev])

columns = ['journal', 'mean', 'median', 'stdev']
j_cost_stats = pd.DataFrame(data, columns=columns)
j_cost_stats

,journal,mean,median,stdev
0,psychological medicine,1617.000000,1917.000,800.416642
1,biomacromolecules,2381.040000,2381.040,NaN
2,journal of medicinal chemistry,1052.490769,1006.720,548.723422
3,journal organic chemistry,685.880000,685.880,NaN
4,journal of proteome research,1844.915000,1844.915,739.683191
5,molecular pharmacology,649.330000,649.330,NaN
6,acs chemical biology,1418.186000,1294.590,507.309560
7,journal of chemical information and modeling,1329.690000,1329.690,NaN
8,biochemistry,665.640000,665.640,NaN
9,gastroenterology,1233.040000,1233.040,1407.085926
